# 1. Import Package

In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from konlpy.tag import Mecab
from tqdm import tqdm

import numpy as np
import json
import pandas as pd
import ast
import os
import re

from osAdvanced import File_Control

import subprocess

# 2. 데이터 전처리

## 2.1 json 파일을 불러온뒤 데이터 로드하기
데이터는 AIHUB의 논문자료 요약 소개 데이터(https://aihub.or.kr/aidata/30712)임.<br>

In [ ]:
DATA_PATH = "" # 데이터 경로 지정

data_paths = File_Control.searchAllFilesInDirectory(DATA_PATH, "json", )

data_list = []

for path in data_paths:
    
    with open(path, 'r') as f:
        json_data = json.load(f)
        for paper_info in json_data['data']:
            
            data_dict = {}     
            for summary_entire in paper_info['summary_entire']:
                #data_dict['news'] = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z]', '', summary_entire['orginal_text'])
                #data_dict['summary'] = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z]', '', summary_entire['summary_text'])          
                data_dict['news'] = summary_entire['orginal_text']
                data_dict['summary'] = summary_entire['summary_text']
            data_list.append(data_dict)

## 2.2 불러온 데이터의 결측치 제거 및 Pandas Dataframe으로 저장
모델의 max length가 512이기에, summary 값의 길이가 512 이상인 값은 제거함.

In [ ]:
print("# Loaded Data : ", len(data_list))
na = ["", "NAME", "원문 참고", "ㅊ", "\n", "\t", "#VALUE!", "#DIV/0!", "#N/A", "#NULL!", "#REF!", "#NUM!", "#NAME!"]

for s in data_list:
    if len(s['summary']) > 512:
        print(f"Dropped : \
                news : {s['news'][:4] + '...'} \t| \
                summary : {s['summary'][:9] + '...'}")
        data_list.remove(s)
    if (s['news'] in na) or (s['summary'] in na):
        print(f"Dropped : \
              news : {s['news'] if len(s['news']) < 10 else s['news'][:4] + '...'} \t| \
              summary : {s['summary'] if len(s['summary']) < 10 else s['summary'][:9] + '...'}")
        data_list.remove(s)
print("# N/A Drop Complete : ", len(data_list))
        
df = pd.DataFrame(data_list)
df = df.dropna(axis='index', inplace=False)
df.reset_index(drop=True, inplace=True)

## 2.3 데이터 확인

In [ ]:
df.head()

In [ ]:
df.describe()

## 2.4 Test/Train Set 분할

In [ ]:
SAVE_PATH = "./data"
np.random.seed(1)
msk = np.random.rand(len(df)) < 0.7

train_df = df[msk]
test_df = df[~msk]

In [ ]:
df.to_csv(SAVE_PATH + "/preprocessed_kobart.tsv", sep='\t')
train_df.to_csv(SAVE_PATH+ "/preprocessed_kobart_train.tsv", sep='\t')
test_df.to_csv(SAVE_PATH + "/preprocessed_kobart_test.tsv", sep='\t')

# 3. Model Train

## 3.1 Model Training
기존의 데이터는 https://dacon.io/competitions/official/235673/overview/description(한국어 문서 생성요약 AI 경진대회)로 학습됨.<br>
기존의 데이터로 pretrained된 모델을 AIHUB의 데이터로 Fine-Tuning 시킴.
<br>
<br>
Epoch은 3으로 지정,<br>
test데이터와 train데이터는 https://aihub.or.kr/aidata/30712(논문자료 요약 소개)를 전처리한 데이터로 설정함. <br>
<br>
- 미리 학습된 모델/파라미터가 ./kobart_summary_pretrained 디렉토리에 파일로 존재해야함.

In [ ]:
#train_finetuning.train(train_file="./data/preprocessed_0615_kobart_train.tsv", test_file="./data/preprocessed_0615_kobart_test.tsv")
subprocess.call("python train_finetuning.py --gradient_clip_val 1.0 \
                --max_epochs 3 \
                --default_root_dir logs_finetuned \
                --gpus 1 \
                --batch_size 32 \
                --num_workers 4 \
                --test_file ./data/preprocessed_kobart_test.tsv \
                --train_file ./data/preprocessed_kobart_train.tsv ", shell=True)

## 3.2 Model Binary 만들기


In [ ]:
#train_finetuning.train(train_file="./data/preprocessed_0615_kobart_train.tsv", test_file="./data/preprocessed_0615_kobart_test.tsv")
subprocess.call("python get_model_binary.py --hparams ./logs_finetuned/tb_logs/default/version_0/hparams.yaml --model_binary ./logs_finetuned/last.ckpt", shell=True)

## 3.3 Streamlit으로 모델 Demo Web Server 생성

In [ ]:
subprocess.call("streamlit run infer.py", shell=True)